## Setup

Install the required dependencies for data analysis.

In [ ]:
!pip install numpy>=1.24.0 pandas>=2.0.0 matplotlib>=3.7.0

## 1. Load the Data

Read the sales data from the CSV file.

In [ ]:
import pandas as pd

# Download the sales data directly from GitHub (data branch)
url = 'https://raw.githubusercontent.com/jcanizalez/pytorch-time-series-forecasting/main/data/sales.csv'
df = pd.read_csv(url)

# Preview the first five rows
df.head()

,date,store_id,cat_id,sales
0,2011-01-29,TX_1,FOODS,3950.35
1,2011-01-30,TX_1,FOODS,3844.97
2,2011-01-31,TX_1,FOODS,2888.03
3,2011-02-01,TX_1,FOODS,3631.28
4,2011-02-02,TX_1,FOODS,3072.18


## 2. Data Overview

Verify the number of unique stores and product categories in the dataset.

In [ ]:
print(f"Unique stores: {df['store_id'].nunique()}")
print(f"Unique categories: {df['cat_id'].nunique()}")

Unique stores: 10
Unique categories: 3


## 3. Date Range Analysis

Convert the date column to datetime format and verify the date range of the sales data.

In [ ]:
df['date'] = pd.to_datetime(df['date'])
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

Date range: 2011-01-29 00:00:00 to 2016-05-22 00:00:00


## 4. Check for Duplicates

Identify any duplicated date entries within each store and category combination.

In [ ]:
duplicated_mask = df.duplicated(subset=['store_id', 'cat_id', 'date'], keep=False)
num_duplicates = duplicated_mask.sum()
print(f"Number of duplicated date entries: {num_duplicates}")
if num_duplicates > 0:
    print("\nSample duplicates:")
    print(df[duplicated_mask].head(10))

Number of duplicated date entries: 0


## 5. Check for Missing Dates

Verify if there are any missing dates in the time series for each store-category combination.

In [ ]:
date_range = pd.date_range(start=df['date'].min(), end=df['date'].max(), freq='D')
expected_records = len(date_range) * df['store_id'].nunique() * df['cat_id'].nunique()
actual_records = len(df)
missing_records = expected_records - actual_records

print(f"Expected records (complete date range): {expected_records}")
print(f"Actual records: {actual_records}")
print(f"Missing records: {missing_records}")

# Check for missing dates per store-category combination
if missing_records > 0:
    for store in df['store_id'].unique():
        for cat in df['cat_id'].unique():
            subset = df[(df['store_id'] == store) & (df['cat_id'] == cat)]
            subset_dates = set(subset['date'])
            all_dates = set(date_range)
            missing_dates = all_dates - subset_dates
            if len(missing_dates) > 0:
                print(f"\n{store} - {cat}: {len(missing_dates)} missing dates")
                print(f"Sample missing dates: {sorted(missing_dates)[:5]}")
                break
        if len(missing_dates) > 0:
            break
else:
    print("\n✓ No missing dates found!")

Expected records (complete date range): 58230
Actual records: 58230
Missing records: 0

✓ No missing dates found!


## 6. Outlier Detection

Identify outliers in the sales data using the Interquartile Range (IQR) method.

In [ ]:
Q1 = df['sales'].quantile(0.25)
Q3 = df['sales'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['sales'] < lower_bound) | (df['sales'] > upper_bound)]

print(f"Q1 (25th percentile): ${Q1:.2f}")
print(f"Q3 (75th percentile): ${Q3:.2f}")
print(f"IQR: ${IQR:.2f}")
print(f"Lower bound: ${lower_bound:.2f}")
print(f"Upper bound: ${upper_bound:.2f}")
print(f"\nNumber of outliers: {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)")

if len(outliers) > 0:
    print("\nSample outliers:")
    print(outliers[['date', 'store_id', 'cat_id', 'sales']].head(10))

Q1 (25th percentile): $1342.25
Q3 (75th percentile): $4641.09
IQR: $3298.85
Lower bound: $-3606.02
Upper bound: $9589.36

Number of outliers: 1455 (2.50%)

Sample outliers:
           date store_id cat_id     sales
1598 2015-06-15     TX_1  FOODS  10036.83
6195 2012-02-05     TX_2  FOODS   9683.29
6251 2012-04-01     TX_2  FOODS   9969.98
6257 2012-04-07     TX_2  FOODS  10122.12
6265 2012-04-15     TX_2  FOODS   9692.91
6286 2012-05-06     TX_2  FOODS  10007.18
6748 2013-08-11     TX_2  FOODS   9699.05
6776 2013-09-08     TX_2  FOODS   9604.35
6783 2013-09-15     TX_2  FOODS  10379.24
6804 2013-10-06     TX_2  FOODS   9649.17


## 7. Check for Missing Values

Identify any missing values in the sales column.

In [ ]:
missing_sales = df['sales'].isna().sum()
print(f"Missing sales values: {missing_sales}")

if missing_sales > 0:
    print(f"Percentage of missing values: {missing_sales/len(df)*100:.2f}%")
    print("\nSample records with missing sales:")
    print(df[df['sales'].isna()][['date', 'store_id', 'cat_id', 'sales']].head(10))
else:
    print("✓ No missing sales values!")

Missing sales values: 0
✓ No missing sales values!


## 8. Data Cleaning

Replace outliers with null values and impute missing data using linear interpolation.

In [ ]:
df_cleaned = df.copy()

# Set outliers to null
df_cleaned.loc[(df_cleaned['sales'] < lower_bound) | (df_cleaned['sales'] > upper_bound), 'sales'] = None

# Count nulls after setting outliers to null
nulls_before_imputation = df_cleaned['sales'].isna().sum()
print(f"Total null values (outliers + original missing): {nulls_before_imputation}")

# Sort by store, category, and date to ensure proper interpolation
df_cleaned = df_cleaned.sort_values(['store_id', 'cat_id', 'date'])

# Interpolate missing values within each store-category group
df_cleaned['sales'] = df_cleaned.groupby(['store_id', 'cat_id'])['sales'].transform(
    lambda x: x.interpolate(method='linear', limit_direction='both')
)

# Check remaining nulls after interpolation
nulls_after_imputation = df_cleaned['sales'].isna().sum()
print(f"Null values after interpolation: {nulls_after_imputation}")
print(f"Successfully imputed: {nulls_before_imputation - nulls_after_imputation} values")

# Display sample of imputed data
print("\nSample of cleaned data:")
print(df_cleaned[['date', 'store_id', 'cat_id', 'sales']].head(10))

Total null values (outliers + original missing): 1455
Null values after interpolation: 0
Successfully imputed: 1455 values

Sample of cleaned data:
            date store_id cat_id    sales
17469 2011-01-29     CA_1  FOODS  7240.65
17470 2011-01-30     CA_1  FOODS  6705.51
17471 2011-01-31     CA_1  FOODS  4584.85
17472 2011-02-01     CA_1  FOODS  4965.46
17473 2011-02-02     CA_1  FOODS  4368.07
17474 2011-02-03     CA_1  FOODS  5247.42
17475 2011-02-04     CA_1  FOODS  5820.45
17476 2011-02-05     CA_1  FOODS  8551.46
17477 2011-02-06     CA_1  FOODS  7435.86
17478 2011-02-07     CA_1  FOODS  5535.84


## 9. Save Processed Data

Save the cleaned and preprocessed data to a CSV file.

In [ ]:
import os

# Create data directory if it doesn't exist
os.makedirs('../data', exist_ok=True)

output_path = '../data/sales_processed.csv'
df_cleaned.to_csv(output_path, index=False)
print(f"✓ Preprocessed data saved to {output_path}")
print(f"Total records saved: {len(df_cleaned)}")

✓ Preprocessed data saved to ../data/sales_processed.csv
Total records saved: 58230
